In [ ]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))
                
def transform_id_x(x, y):
    return x
def transform_id_y(x, y):
    return y

In [ ]:
from spinorama.load import graph_melt
import flammkuchen as fl
from spinorama.graph_contour import compute_contour
from spinorama.graph import contour_params_default, graph_contour_common

df_all = fl.load('../cache.parse_all_speakers.h5')

In [ ]:
df = df_all['Genelec 8341A']['ASR']['asr']
print(df.keys())

In [ ]:
dfu = df['SPL Vertical_unmelted']
af, am, az = compute_contour(dfu.loc[dfu.Freq>100])
print(len(af[0]), len(am[:,0]), az.shape )
print(af[0][0:3])
print(am[:,0])

In [ ]:
graph_width = 1024
from spinorama.graph_isobands import find_isobands
graph_height = 720

def transform_log(x, y):
    return 6*np.log10(x)*graph_width/776

def transform_radian(x, y):
    return 1.325*y/180*math.pi*graph_height/360

bands = [-72, -18, -15, -12, -9, -6, -3, +3]
df_iso = find_isobands(af, am, az.T, bands, transform_log, transform_radian)

color_legend = ['[{0}, {1}]'.format(bands[i], bands[i+1]) for i in range(0, len(bands)-1)] + ['>{0}'.format(bands[-1])]
color_range = ['black', 'blue', 'steelblue', 'green', 'yellow', 'orange', 'red', 'white']

print(color_legend)

isobands = alt.Chart(alt.Data(values=df_iso['features'])
).mark_geoshape(
    stroke='red',
    strokeWidth=0
).encode(
    # alt.Color('properties.z_low:Q', scale=alt.Scale(scheme='inferno'), title = "IsoBands")
    alt.Color('properties.z_low:O', 
              scale=alt.Scale(zero=False, domain=bands, range=color_range), 
              legend=alt.Legend(orient='bottom', title='Relative dB SPL'))
).project('identity'
).properties(
    width=graph_width,
    height=graph_height
)

In [ ]:
empty = pd.DataFrame({'Freq', 'Angle'})
yTicks = [-180+30*i for i in range(0,13)]
print(yTicks)
legends = alt.Chart(empty).mark_point(clip=True).encode(
    x=alt.X('x:Q', scale=alt.Scale(type='log', nice=False, domain=[20, 20000]), title='Frequency (Hz)'),
    y=alt.Y('y:Q', scale=alt.Scale(nice=False, domain=[-180, 180]), axis=alt.Axis(format='.0d', values=yTicks, title='Angle')), 
).properties(
    width=graph_width,
    height=graph_height
)

In [ ]:
isobands + legends

In [ ]:
def transform_polar_r(x, y):
    a = y*math.pi/180
    return np.log10(x/20)*np.cos(a)

def transform_polar_theta(x, y):
    a = y*math.pi/180
    return np.log10(x/20)*np.sin(a)


df_polar = find_isobands(af, am, az.T, bands, transform_polar_r, transform_polar_theta, False, False)
polar = alt.Chart(alt.Data(values=df_polar['features'])
).mark_geoshape(
    strokeWidth=0
).encode(
    alt.Color('properties.z_low:O', 
              scale=alt.Scale(zero=False, domain=bands, range=color_range), 
              legend=alt.Legend(orient='bottom', title='Relative dB SPL'))
).project('identity'
).properties(
    width=graph_width-40,
    height=graph_height-40
)

In [ ]:
# polar

In [ ]:
from spinorama.graph_radar import grid_grid, grid_circle, grid_text

In [ ]:
anglelist = [a for a in range(180, -180, -30)]

In [ ]:
grid_df = grid_grid(anglelist)
circle_df, circle_text = grid_circle(anglelist, 100)
text_df = grid_text(anglelist)

In [ ]:
grid = alt.Chart(grid_df).mark_line(
    color='#FFFFFF'
).encode(
    alt.Latitude('x:Q'), 
    alt.Longitude('y:Q'), 
    size=alt.value(1),
    opacity=alt.value(0.5)
).project(type='azimuthalEquidistant', rotate=[0, 0, 270])

circle = alt.Chart(circle_df).mark_line(
    color='#FFFFFF'
).encode(
    alt.Latitude('x:Q'),
    alt.Longitude('y:Q'),
    alt.Color('value:Q', legend=None),
    size=alt.value(0.5),
    opacity=alt.value(0.5)
).project(
    type='azimuthalEquidistant',
    rotate=[0, 0, 270]
)
    
legend = alt.Chart(circle_text).mark_text(
        dx=15,
        dy=-10
    ).encode(
        alt.Latitude('x:Q'), 
        alt.Longitude('y:Q'), 
        text='text:O'
    ).project(type='azimuthalEquidistant', rotate=[0, 0, 270])
        
text = alt.Chart(text_df).mark_text(
).encode(
    alt.Latitude('x:Q'), 
    alt.Longitude('y:Q'), 
    text='text:O'
).project(type='azimuthalEquidistant', rotate=[0, 0, 270])


In [ ]:
polar+grid+circle+legend+text